In [2]:
import pandas as pd

In [24]:
data = pd.read_csv('WMT_Grocery_202209.csv')

/var/folders/9c/67cc_3jj62d2ds4cnt_766t80000gn/T/ipykernel_5068/3266408272.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('WMT_Grocery_202209.csv')


In [25]:
data = data[["index", "CATEGORY", "PRODUCT_NAME", "PRICE_CURRENT", "PRODUCT_SIZE"]]
data 

,index,CATEGORY,PRODUCT_NAME,PRICE_CURRENT,PRODUCT_SIZE
0,0,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz",2.67,10
1,1,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz",2.67,10
2,2,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz",2.67,10
3,3,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz",2.67,10
4,4,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz.",3.12,12
...,...,...,...,...,...
568529,568529,Wine,Farm Fresh Blueberry Moscato 750ml,9.98,750
568530,568530,Wine,Farm Fresh Peach Moscato 750 Ml,9.98,750
568531,568531,Wine,Farm Fresh Raspberry Moscato 750ml,9.98,750
568532,568532,Wine,Farm Fresh Mango Moscato 750ml,9.98,750


In [26]:
data = data.rename(columns={"index": "id", "CATEGORY": "jenis_barang_id", "PRODUCT_NAME": "nama", "PRICE_CURRENT": "harga", "PRODUCT_SIZE": "jumlah_stok"})

In [27]:
category = pd.DataFrame(set(data["jenis_barang_id"]))

In [28]:
category = category.reset_index().rename(columns={0: "kategori", "index":"id"})
category

,id,kategori
0,0,Popcorn
1,1,Frozen Meat & Seafood
2,2,Chips
3,3,Beverage Deals
4,4,Sugars & Sweeteners
...,...,...
109,109,Organic Produce
110,110,Fresh Vegetables
111,111,International foods
112,112,Hot Cereals


In [29]:
def addOneId(row):
    return row["id"] + 1

category["id"] = category.apply(addOneId, axis=1)

In [30]:
category.to_csv("kategori.csv", index=False)

In [31]:
category

,id,kategori
0,1,Popcorn
1,2,Frozen Meat & Seafood
2,3,Chips
3,4,Beverage Deals
4,5,Sugars & Sweeteners
...,...,...
109,110,Organic Produce
110,111,Fresh Vegetables
111,112,International foods
112,113,Hot Cereals


In [32]:
def tansformData(row):
    row['harga'] = row['harga'] * 10000
    row['id'] = row['id'] + 1

    return row

data = data.apply(tansformData, axis=1)

In [35]:
data["harga"] = data["harga"].astype(int)

In [36]:
category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        114 non-null    int64 
 1   kategori  114 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [37]:
category = category.to_dict("records")
def changeCategory(row):

    for cat in category:
        if row["jenis_barang_id"] == cat["kategori"]:
            row["jenis_barang_id"] = str(cat["id"])
            break
    
    return row

data = data.apply(changeCategory, axis=1)

In [38]:
data.to_csv("barang.csv", index=False)

In [62]:
items = pd.read_csv("barang.csv")
category = pd.read_csv("kategori.csv")


In [63]:
itemsFinal = pd.DataFrame()
category = category.to_dict("records")
for row in category:
    cat = row["id"]

    temp = items[items.jenis_barang_id == cat][:10]

    itemsFinal = pd.concat([itemsFinal,temp])

    

In [66]:
itemsFinal

,id,jenis_barang_id,nama,harga,jumlah_stok
0,0,1,"ACT II Xtreme Butter Microwave Popcorn, 2.75 O...",65700,2.75
1,1,1,"Smartfood White Cheddar Popcorn, 6.75 oz Bag",42800,6.75
2,2,1,"Great Value Extra Butter Microwave Popcorn, 2....",105300,2.55
3,3,1,"Smartfood Popcorn, White Cheddar, 9.75 oz",49800,9.75
4,4,1,"ACT II Butter Lovers Microwave Popcorn, 2.75 O...",65700,2.75
...,...,...,...,...,...
1135,1135,114,"Freshness Guaranteed Cheese Danish, 14 oz, 8 s...",39800,14
1136,1136,114,"Freshness Guaranteed Apple Turnovers, 4 count,...",39800,10
1137,1137,114,"Freshness Guaranteed Mini Iced Cinnamon Rolls,...",39800,13
1138,1138,114,"Assorted Sweet Rolls, 24 oz, 6 Rolls",39800,24


In [65]:
itemsFinal = itemsFinal.drop("id", axis=1).reset_index(drop=True).reset_index().rename(columns={"index": "id"})

In [67]:
def addOneId(row):
    return row["id"] + 1

itemsFinal["id"] = itemsFinal.apply(addOneId, axis=1)

In [69]:
itemsFinal.to_csv("barang.csv", index=False)

PUSH DATA TO DATABASE

In [75]:
produk = pd.read_csv("barang.csv")

produk.dropna(inplace=True)

produk.to_csv("barang.csv", index=False)

In [76]:
import requests
import math
from tqdm.auto import tqdm

tqdm.pandas()
url = "http://localhost:3000/barang"

def reqInsert(row):
    try:
        float(row["jumlah_stok"])
    except:
        row["jumlah_stok"] = "0"

    myobj = {
        "id": row["id"],
        "nama": row["nama"],
        "harga": row["harga"],
        "jumlah_stok": int(math.floor(float(row["jumlah_stok"]))),
        "jenis_barang_id": row["jenis_barang_id"]
    }

    requests.post(url, json = myobj)

produk.progress_apply(reqInsert, axis=1)

100%|██████████| 994/994 [00:02<00:00, 400.61it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
989    None
990    None
991    None
992    None
993    None
Length: 994, dtype: object

In [73]:
kategori = pd.read_csv("kategori.csv")

In [74]:
import requests

url = "http://localhost:3000/jenis-barang"

def reqInsert(row):
    myobj = {
        "id": row["id"],
        "kategori": row["kategori"]
    }

    requests.post(url, json = myobj)

kategori.apply(reqInsert, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
109    None
110    None
111    None
112    None
113    None
Length: 114, dtype: object

In [38]:
member = pd.read_csv("member.csv")

In [16]:
member

NameError: name 'member' is not defined

In [18]:
import requests

url = "http://localhost:3000/member"

def reqInsert(row):
    myobj = {
        "id": row["id"],
        "nama": row["nama"],
        "no_telepon": row["no_telepon"],
        "poin": row["poin"]
    }

    requests.post(url, json = myobj)

member.apply(reqInsert, axis=1)

NameError: name 'member' is not defined

In [13]:
pegawai = pd.read_csv("pegawai.csv")

In [14]:
pegawai

,id,nama,bidang_kerja_id,email,password
0,1,Madeline Moses,1,madelinemoses@gmail.com,madeline123
1,2,Marny Nixon,2,marnynixon3989@gmail.com,marrnycantik
2,3,Karen Winters,2,karenwinters@gmail.com,admin123
3,4,Ferris Norton,1,ferrisnorton@gmail.com,rabusehat
4,5,Cynthia Walls,1,cynthiawalls8226@gmail.com,mantanterindah
5,6,Lareina Livingston,1,lareinalivingston@gmail.com,ibusayang
6,7,Janna Travis,1,jannatravis@gmail.com,kitakita
7,8,Oleg Little,1,oleglittle8978@gmail.com,oleg23
8,9,Islam Daniel,1,islamdaniel@gmail.com,Islamdamai
9,10,Hanna Wilkins,1,hannawilkins202@gmail.com,wikipedia123


In [15]:
import requests

url = "http://localhost:3000/pegawai/register"

def reqInsert(row):
    myobj = {
        "id": row["id"],
        "nama": row["nama"],
        "bidang_kerja_id": row["bidang_kerja_id"],
        "email": row["email"],
        "password": row["password"]
    }

    print(requests.post(url, json = myobj))

pegawai.apply(reqInsert, axis=1)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
dtype: object

In [8]:
bidangKerja = pd.read_csv("bidang-kerja.csv")


In [11]:
bidangKerja

,id,pekerjaan
0,1,manager
1,2,Woker


In [12]:
import requests

url = "http://localhost:3000/bidang-kerja"

def reqInsert(row):
    myobj = {
        "id": row["id"],
        "pekerjaan": row["pekerjaan"],
    }

    print(requests.post(url, json = myobj))

bidangKerja.apply(reqInsert, axis=1)

<Response [201]>
<Response [201]>


0    None
1    None
dtype: object